In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy.core.defchararray as npchar

In [94]:
# set initial properties for data location
res = 40000
data_location = "matrices/"
true_location = "trueTADs/"
armatus_location = "./armatus"
out_location = "yielded/"
noise_values = [4, 8, 12, 16, 20]
sim_values = list(range(1, 6))
TAD_callers = ("lava_modularity", "lava_armatus")

In [101]:
# set range of gammas for methods
armatus_gamma = [i / 2 for i in range(11)]
modularity_gamma = list(range(101))

In [60]:
# custom functions
vector_str = np.vectorize(str)

def TAD_bins(arr1, arr2):
    vector_str = np.vectorize(str)
    return npchar.add(vector_str(arr1), npchar.add(",", vector_str(arr2)))

def TAD_boundaries(arr1, arr2):
    return np.unique(np.append(arr1, arr2))

In [8]:
# load true TADs coordinates
true_TADs = pd.DataFrame(index=noise_values, columns=sim_values)
for noise in noise_values:
    for sim in sim_values:
        print(f"\rnoise={noise} sim={sim}", end="")
        true_TADs.loc[noise, sim] = np.loadtxt(f"{true_location}simHiC_TADintervals_coords_noise{noise}_sim{sim}.txt")
print("\nfinished")

 noise=20 sim=5
finished


In [34]:
# combine true TADs coordinates into strings
true_TADs_bins = pd.DataFrame(index=noise_values, columns=sim_values)
for noise in noise_values:
    for sim in sim_values:
        true_TADs_bins.loc[noise, sim] = npchar.add(vector_str(true_TADs.loc[noise, sim][:, 0]), npchar.add(",", vector_str(true_TADs.loc[noise, sim][:, 1])))

In [35]:
# make unique true TADs boundaries
true_TADs_boundaries = pd.DataFrame(index=noise_values, columns=sim_values)
for noise in noise_values:
    for sim in sim_values:
        true_TADs_boundaries.loc[noise, sim] = np.unique(np.append(true_TADs.loc[noise, sim][:, 0], true_TADs.loc[noise, sim][:, 1]))

In [115]:
TAD_stats = dict.fromkeys(("TPR_bins", "TPR_boundaries", "FDR_bins", "FDR_boundaries"))
method_ranges = {"armatus":armatus_gamma, "lava_modularity":modularity_gamma, "lava_armatus":armatus_gamma}
for value in TAD_stats.keys():
    for method in TAD_callers:
        TAD_stats[value] = dict.fromkeys(TAD_callers)
        TAD_stats[value][method] = dict.fromkeys(method_ranges[method])
        for gamma in TAD_stats[value][method].keys():
            TAD_stats[value][method][gamma] = pd.DataFrame(index=noise_values, columns=sim_values)
        print(value, method, TAD_stats[value][method])

TPR_bins lava_modularity {0:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 1:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 2:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 3:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 4:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 5:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN

TPR_boundaries lava_modularity {0:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 1:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 2:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 3:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 4:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 5:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  Na

FDR_bins lava_modularity {0:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 1:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 2:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 3:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 4:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 5:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN

FDR_boundaries lava_modularity {0:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 1:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 2:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 3:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 4:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN, 5:       1    2    3    4    5
4   NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  Na

In [114]:
TAD_stats["TPR_bins"]["lava_modularity"]

In [77]:
# count TPR and FDR for TADs
for method in TAD_callers:
    for noise in noise_values:
        for sim in sim_values:
            for gamma in method_ranges[method]:
                TADs = np.loadtxt(f"{out_location}{method}_noise{noise}_sim{sim}_gamma{gamma}.txt", ndmin=2)
                bins = TAD_bins(TADs[:, 0], TADs[:, 1])
                boundaries = TAD_boundaries(TADs[:, 0], TADs[:, 1])
                TAD_stats["TPR_bins"][method][gamma].loc[noise, sim] = sum(np.isin(bins, true_TADs_bins.loc[noise, sim]))
                TAD_stats["FDR_bins"][method][gamma].loc[noise, sim] = sum(~np.isin(bins, true_TADs_bins.loc[noise, sim]))
                TAD_stats["TPR_boundaries"][method][gamma].loc[noise, sim] = sum(np.isin(boundaries, true_TADs_boundaries.loc[noise, sim]))
                TAD_stats["FDR_boundaries"][method][gamma].loc[noise, sim] = sum(~np.isin(boundaries, true_TADs_boundaries.loc[noise, sim]))

TypeError: 'NoneType' object is not subscriptable